In [2]:
import geopandas as gpd

In [3]:
path_to_shapefile = "./concelhos.shp"

In [4]:
shapefile_data_all = gpd.read_file(path_to_shapefile)

In [5]:
shapefile_data_all.head()

,ID_0,ISO,NAME_0,ID_1,NAME_1,ID_2,NAME_2,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
0,182,PRT,Portugal,1,Évora,1,Évora,PT.EV.EV,0,0705,Concelho,Municipality,None,None,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506..."
1,182,PRT,Portugal,1,Évora,2,Alandroal,PT.EV.AL,0,0701,Concelho,Municipality,None,None,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343..."
2,182,PRT,Portugal,1,Évora,3,Arraiolos,PT.EV.AR,0,0702,Concelho,Municipality,None,None,"POLYGON ((-7.88611 38.92495, -7.88580 38.92472..."
3,182,PRT,Portugal,1,Évora,4,Borba,PT.EV.BO,0,0703,Concelho,Municipality,None,None,"POLYGON ((-7.46362 38.92344, -7.46344 38.92329..."
4,182,PRT,Portugal,1,Évora,5,Estremoz,PT.EV.ES,0,0704,Concelho,Municipality,None,None,"POLYGON ((-7.52770 39.00080, -7.52765 39.00066..."


In [6]:
shapefile_data = shapefile_data_all[['NAME_1', 'NAME_2', 'geometry', 'CCA_2']]
shapefile_data.columns = ['District', 'Municipality', 'geometry', 'CCA_2']
shapefile_data.head()

,District,Municipality,geometry,CCA_2
0,Évora,Évora,"POLYGON ((-7.79291 38.76507, -7.79287 38.76506...",0705
1,Évora,Alandroal,"POLYGON ((-7.25937 38.77351, -7.25921 38.77343...",0701
2,Évora,Arraiolos,"POLYGON ((-7.88611 38.92495, -7.88580 38.92472...",0702
3,Évora,Borba,"POLYGON ((-7.46362 38.92344, -7.46344 38.92329...",0703
4,Évora,Estremoz,"POLYGON ((-7.52770 39.00080, -7.52765 39.00066...",0704


In [7]:
shapefile_data['District'].unique()

array(['Évora', 'Aveiro', 'Azores', 'Beja', 'Braga', 'Bragança',
       'Castelo Branco', 'Coimbra', 'Faro', 'Guarda', 'Leiria', 'Lisboa',
       'Madeira', 'Portalegre', 'Porto', 'Santarém', 'Setúbal',
       'Viana do Castelo', 'Vila Real', 'Viseu'], dtype=object)

### Check and fix duplicated municipalities

In [8]:
new_shapefile_data = shapefile_data.copy()

In [9]:
municipalities = shapefile_data['Municipality'].tolist()
for munic in municipalities:
    appearances = municipalities.count(munic)
    if appearances > 1:
        print(munic)

Calheta
Lagoa
Braga
Braga
Alcoutim
Alcoutim
Lagoa
Calheta


From the comments on the website, we know that:
* Braga is reported twice instead of Guimarães
* Alcoutim is reported twice instead of Tavira

In [10]:
shapefile_data.loc[shapefile_data['Municipality'] == 'Braga']

,District,Municipality,geometry,CCA_2
68,Braga,Braga,"POLYGON ((-8.35704 41.61721, -8.35661 41.61719...",0303
69,Braga,Braga,"POLYGON ((-8.31783 41.54539, -8.31739 41.54216...",0308


In [11]:
new_shapefile_data.loc[new_shapefile_data['CCA_2'] == '0308', 'Municipality'] = 'Guimarães'

In [12]:
shapefile_data.loc[shapefile_data['Municipality'] == 'Alcoutim']

,District,Municipality,geometry,CCA_2
121,Faro,Alcoutim,"POLYGON ((-7.56960 37.52726, -7.56915 37.52715...",0802
122,Faro,Alcoutim,"MULTIPOLYGON (((-7.61847 37.11625, -7.61847 37...",0814


In [13]:
new_shapefile_data.loc[new_shapefile_data['CCA_2'] == '0814', 'Municipality'] = 'Tavira'

For the other two, need to check the codes

In [14]:
shapefile_data.loc[shapefile_data['Municipality'] == 'Calheta']

,District,Municipality,geometry,CCA_2
34,Azores,Calheta,"MULTIPOLYGON (((-28.03560 38.60697, -28.03561 ...",4501
183,Madeira,Calheta,"MULTIPOLYGON (((-17.24440 32.78358, -17.24445 ...",3101


In [15]:
shapefile_data.loc[shapefile_data['Municipality'] == 'Lagoa']

,District,Municipality,geometry,CCA_2
37,Azores,Lagoa,"POLYGON ((-25.55903 37.77534, -25.55883 37.775...",4201
126,Faro,Lagoa,"MULTIPOLYGON (((-8.44033 37.08669, -8.44038 37...",0806


These two are not duplicated, two municipalities exist with that name

### Check how Ponte de Sor is reported

In [16]:
new_shapefile_data.loc[new_shapefile_data['CCA_2'] == '1213']

,District,Municipality,geometry,CCA_2
205,Portalegre,Ponte de Sôr,"POLYGON ((-7.98201 39.38449, -7.98185 39.38443...",1213


In [17]:
new_shapefile_data.loc[new_shapefile_data['Municipality'] == 'Ponte de Sôr', 'Municipality'] = 'Ponte de Sor'

### Check how Ponte de Sor is reported

In [18]:
new_shapefile_data.loc[new_shapefile_data['CCA_2'] == '1213']

,District,Municipality,geometry,CCA_2
205,Portalegre,Ponte de Sor,"POLYGON ((-7.98201 39.38449, -7.98185 39.38443...",1213


In [19]:
new_shapefile_data.loc[new_shapefile_data['Municipality'] == 'Ponte de Sôr', 'Municipality'] = 'Ponte de Sor'

### Check how Meda is reported

In [20]:
new_shapefile_data.loc[new_shapefile_data['CCA_2'] == '0909']

,District,Municipality,geometry,CCA_2
143,Guarda,Mêda,"POLYGON ((-7.20808 41.03702, -7.20772 41.03698...",0909


In [21]:
new_shapefile_data.loc[new_shapefile_data['Municipality'] == 'Mêda', 'Municipality'] = 'Meda'

## Check no duplicates of CCA_2 and set as index

In [22]:
any(new_shapefile_data['CCA_2'].value_counts() > 1)

False

In [23]:
new_shapefile_data = new_shapefile_data.set_index('CCA_2')

# Save it as a new shapefile

In [24]:
data_schema = {'geometry': 'Polygon',
               'properties': {'CCA_2': 'str',
                              'District': 'str',
                              'Municipality': 'str'}}

In [25]:
new_shapefile_data.to_file("mod_concelhos.shp", schema=data_schema)

**NOTE**: when the created file is uploaded, CCA_2 is not anymore the index and the name of the column Municipality is changed to Municipali

# Create a version without Madeira and Azores

In [26]:
shapefile_no_mad_az = new_shapefile_data.copy()

In [27]:
mask = (shapefile_no_mad_az['District'] == 'Madeira') | (shapefile_no_mad_az['District'] == 'Azores')
shapefile_no_mad_az = shapefile_no_mad_az.drop(shapefile_no_mad_az.loc[mask].index)

Since do municipalities with the same name are present anymore, we can set the name as the index

In [28]:
shapefile_no_mad_az = shapefile_no_mad_az.reset_index().set_index('Municipality')

In [29]:
shapefile_no_mad_az.to_file("concelhos_no_azores_and_madeira.shp", schema=data_schema)